## UTMCC DataViz Module 20 Team Project¶
### Food Deserts in the Austin, Texas Metro Area

In [2]:
# # Import dependencies
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import tree
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# from sklearn.metrics import balanced_accuracy_score
# from imblearn.metrics import classification_report_imbalanced
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans

# from scipy import stats
# import seaborn as sns
# # import tensorflow as tf
# # from tensorflow import keras
# # from tensorflow.keras import layers
# # from tensorflow.keras import activations
# # Import checkpoint dependencies
# import os
# # from tensorflow.keras.callbacks import ModelCheckpoint
# # from tensorflow.keras.utils import to_categorical

In [3]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pathlib import Path
from collections import Counter
import datetime as dt
import calendar
import random
# from path import Path

# %matplotlib inline
# from matplotlib import style
# style.use('fivethirtyeight')
# import matplotlib.pyplot as plt
# import statistics
# from flask import Flask, jsonify
# from mpl_toolkits.mplot3d import Axes3D
# # from hvplot import hvPlot
# # import hvplot.pandas
# import plotly.express as px

# # Python SQL toolkit and Object Relational Mapper
# import sqlite3
# import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func
# from sqlalchemy import extract

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Installing Spark

import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [43.6 kB]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.

### PostgreSQL on AWS setup

In [5]:
# Postgres driver for Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-16 22:44:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-01-16 22:44:08 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
# Configure settings for AWS RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.c5qcqhh5xq62.us-east-2.rds.amazonaws.com:5432/Module20_food_deserts"
config = {"user":"postgres", 
          "password": "xxxxxxxx", 
          "driver":"org.postgresql.Driver"}

In [7]:
# Start a Spark session with an additional option that adds the driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Importing and reading data files from the AWS S3 Bucket

In [8]:
# Read in data
from pyspark import SparkFiles

url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/food_access_research_atlas.csv"
spark.sparkContext.addFile(url)
food_access_research_atlas_df = spark.read.csv(SparkFiles.get("food_access_research_atlas.csv"), sep=",", header=True, inferSchema=True)

In [9]:
food_atlas_df = food_access_research_atlas_df
food_atlas_df.show(5)

+-----------+-------+-------+-----+-------+-------+-----------------+--------+---------+-----------------+--------------------+-----------------+------------------+--------+---------------+-----------+------------------+--------+-----------+--------+-------------+---------+----------+----------+------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------+-----------+---------------+-----------+---------------+-------------+------------------+-----------+----------------+-----------+----------------+-----------+----------------+-----------+----------------+-----------+---------------+-------------+------------------+-----------+---------------+-----------+---------------+-----------+---------------+-----------+-----------+-----------+------------+-----------+------------+-----------+---------------+-----------+-------------+-----------+-------------+-----------+-------------+----------+-------------+-----------+------------+------

In [10]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/food_desert_austin_censustract.csv"
spark.sparkContext.addFile(url)
fooddesert_austin_censustract_df = spark.read.csv(SparkFiles.get("food_desert_austin_censustract.csv"), sep=",", header=True, inferSchema=True)

In [11]:
fooddesert_austin_censustract_df.show(5)

+-----+-----------+-----+-------+-----+-------+-------+-----------------+--------+-------------------+-----------------+--------------------+-----------------+------------------+--------+---------------+-----------+------------------+--------+-----------+--------+-------------+---------+----------+----------+------------------+------------------+------------------+-----------------+-----------+-----------+-----------+------------------+------------------+------------------+---------------+-----------+-------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+--------------------+-----------+--------------------+-----------------+--------------------+------------------+-------------------+-----------+-------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+------------------+-------------------+------------------+----

In [12]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/census_tract_shapefiles_all.csv"
spark.sparkContext.addFile(url)
census_tract_shapefiles_all_df = spark.read.csv(SparkFiles.get("census_tract_shapefiles_all.csv"), sep=",", header=True, inferSchema=True)

In [13]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/Strategic_Measure_HE.F.2__Number_and_percentage_of_food_insecure_residents_that_are_living_in_proximity_to_a_City-supported_fresh_food_access_point.csv"
spark.sparkContext.addFile(url)
percent_foodinsecure_df = spark.read.csv(SparkFiles.get("Strategic_Measure_HE.F.2__Number_and_percentage_of_food_insecure_residents_that_are_living_in_proximity_to_a_City-supported_fresh_food_access_point.csv"), sep=",", header=True, inferSchema=True)

In [14]:
percent_foodinsecure_df.show(5)

+--------------------+------------------+-----------------------+------------------------+------------------------+
|                Date|      Census Tract|Census Tract Population|Food Insecure Population|Food Insecure Percentage|
+--------------------+------------------+-----------------------+------------------------+------------------------+
|01/01/2020 12:00:...|   Census Tract 10|                   3777|                     780|                   0.206|
|01/01/2020 12:00:...|   Census Tract 11|                   6608|                    1230|                   0.186|
|01/01/2020 12:00:...|   Census Tract 12|                   5513|                     970|                   0.177|
|01/01/2020 12:00:...|Census Tract 13.05|                   5876|                    1210|                   0.207|
|01/01/2020 12:00:...|Census Tract 13.07|                   3747|                     710|                   0.191|
+--------------------+------------------+-----------------------+-------

In [15]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/food_desert.csv"
spark.sparkContext.addFile(url)
food_desert_df = spark.read.csv(SparkFiles.get("food_desert.csv"), sep=",", header=True, inferSchema=True)

In [16]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/food_desertUS.csv"
spark.sparkContext.addFile(url)
food_desertUS_df = spark.read.csv(SparkFiles.get("food_desertUS.csv"), sep=",", header=True, inferSchema=True)

In [17]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/food_atlasMFI.csv"
spark.sparkContext.addFile(url)
food_atlasMFI_df = spark.read.csv(SparkFiles.get("food_atlasMFI.csv"), sep=",", header=True, inferSchema=True)

In [18]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/LILATracts_1And10_aus.csv"
spark.sparkContext.addFile(url)
LILATracts_1And10_aus_df = spark.read.csv(SparkFiles.get("LILATracts_1And10_aus.csv"), sep=",", header=True, inferSchema=True)

In [19]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/food_austinMFI.csv"
spark.sparkContext.addFile(url)
food_austinMFI_df = spark.read.csv(SparkFiles.get("food_austinMFI.csv"), sep=",", header=True, inferSchema=True)

### Writing to the PostgrSQL database

In [20]:
# Write  _df to table in AWS RDS
food_atlas_df.write.jdbc(url=jdbc_url, table='food_access_research_atlas', mode=mode, properties=config)

In [21]:
# Write  _df to table in AWS RDS
fooddesert_austin_censustract_df.write.jdbc(url=jdbc_url, table='fooddesert_austin_censustract', mode=mode, properties=config)

In [22]:
# Write  _df to table in AWS RDS
percent_foodinsecure_df.write.jdbc(url=jdbc_url, table='percent_foodinsecure', mode=mode, properties=config)

In [37]:
# Write  _df to table in AWS RDS
food_atlasMFI_df.write.jdbc(url=jdbc_url, table='food_atlasMFI', mode=mode, properties=config)

In [23]:
# Write  _df to table in AWS RDS
LILATracts_1And10_aus_df.write.jdbc(url=jdbc_url, table='LILATracts_1And10_aus', mode=mode, properties=config)

In [24]:
# Write  _df to table in AWS RDS
food_austinMFI_df.write.jdbc(url=jdbc_url, table='food_austinMFI', mode=mode, properties=config)

## Data Preprocessing

In [25]:
# Rename column GEOID to CensusTract
census_tract_shapefiles_all_df = census_tract_shapefiles_all_df.withColumnRenamed("GEOID", "CensusTract")
census_tract_shapefiles_all_df.show(5)

+-----------+----------+------------+-------------+--------------+
|CensusTract|PointOrder|SubPolygonId|PointLatitude|PointLongitude|
+-----------+----------+------------+-------------+--------------+
| 1001020100|         1|           1|      32.4988|       -86.482|
| 1001020100|         2|           1|      32.4915|      -86.4787|
| 1001020100|         3|           1|      32.4764|      -86.4812|
| 1001020100|         4|           1|       32.466|      -86.4751|
| 1001020100|         5|           1|      32.4597|      -86.4753|
+-----------+----------+------------+-------------+--------------+
only showing top 5 rows



In [26]:
# Join two of the DataFrames
fooddesert_austin_censusshapes_df = fooddesert_austin_censustract_df.join(census_tract_shapefiles_all_df, on="CensusTract", how="inner")
fooddesert_austin_censusshapes_df.show(5)

+-----------+-----+-----+-------+-----+-------+-------+-----------------+--------+--------+-----------------+--------------------+-----------------+------------------+--------+---------------+-----------+------------------+--------+-----------+--------+-------------+---------+----------+----------+------------------+------------------+------------------+---------+-----------+-----------+----------+-----------------+--------------+------------------+---------------+-----------+-------------------+------------------+------------------+------------------+----------------+------------------+----------------+-----------+--------------------+-----------+--------------------+-----------------+---------------+------------------+------------------+-----------+---------------+-----------+---------------+-----------+---------------+-----------+-----------+---------+-------------------+-----------+------------+-----------+---------------+-----------+------------------+-----------+-------------+---

In [27]:
# Write  _df to table in AWS RDS
fooddesert_austin_censusshapes_df.write.jdbc(url=jdbc_url, table='fooddesert_austin_censusshapes', mode=mode, properties=config)